In [3]:
from typing import Dict, Tuple, Union, List, Any
from fuzzingbook.Fuzzer import RandomFuzzer

In [4]:
random_fuzzer = RandomFuzzer()
random_fuzzer.fuzz()

"6/0'#2#$762 40'5?(;0 4'5(:#!/>  -%?-*?5<*>3 5'7!#%6#964$04'!4/;.>7<></0"

In [5]:
print(RandomFuzzer.__init__.__doc__)

Produce strings of `min_length` to `max_length` characters
           in the range [`char_start`, `char_start` + `char_range`)


In [6]:
random_fuzzer = RandomFuzzer(min_length=10, max_length=20, char_start=97, char_range=26)
random_fuzzer.fuzz()

'fyjdivseknsa'

In [7]:
from fuzzingbook.Fuzzer import PrintRunner

print_runner = PrintRunner()
random_fuzzer.run(print_runner)

rkuslwbawv


('rkuslwbawv', 'UNRESOLVED')

In [8]:
from fuzzingbook.Fuzzer import ProgramRunner

cat = ProgramRunner("cat")
random_fuzzer.run(cat)

(CompletedProcess(args='cat', returncode=0, stdout='iivzzfzkxcijevjmg', stderr=''),
 'PASS')

In [ ]:
import random


def fuzzer(max_length: int = 100, char_start: int = 32, char_range: int = 32) -> str:
    string_length = random.randrange(0, max_length + 1)
    out = ""
    for _ in range(string_length):
        out += chr(random.randrange(char_start, char_start + char_range))
    return out

In [10]:
fuzzer()

'/44;3$('

In [11]:
fuzzer(1000, ord("a"), 26)

'^te!KD\x1c77j6Irc\x1bG[\x1a(YCOnLY^f?tCl\x1dvPL39os!)xngH!K0Y\x1b18xjRxWK%26xqBL.[![c+IXZkJV37b\'8/7^p3jN[AGb&x`$2Y* j>z&rR8D\x1b?+8G?LS7Xd(CvN!eMZX@4\x1b.*:B^DO4f+0qA!y/xR^z%M>5$,9\'S-\x1dctRIEiI8h]mf/ze-H;VR\x1d!<J3- Ww8SIEs0sp\'D_9$$\x1cJxpbA:D8NKG_DXIp\x1a#"R\x1c86>tW8y\'gNm2o77`f0""yD\x1a@o7\x1e)s7WhL8>\x1bf9 6%sa%68Tx[O&f5ei<cp<D)CTGD!?\x1c8"WdPUs,oHor_qfz3:a,"XJWOI\x1a4oALI^T%);L^b2aqGg`\x1d"2\x1cVOi!gTiar&/s,-7\\WiFY^\x1f-Uh:kn^(\x1f*Lw\x1b[zDIOP^E^J4HwxTq*@<v-Z>vN:L;Mh\x1dRk!Tk3]*xnMJcU#LH!TP\x1e $a\x1d <\'=oGum^A-A%UW"\x1f/TxRWr2\x1a\x1aj!U(,.o_K00K5 v`GQe9KaZvil]5#p9Q\x1cB\x1d\x1bly\x1aO;l=t-!=(O+^exjIU(\x1eNQ)1\x1a8>?6G*OZRP\x1b,Qex!BDFvd:CKZr\x1eJ*kNhZKa\x1fMt9%6v#78J`_3TmP^*/WJmNm$[u\'Y,m!?]3\x1a8MYHqGv9Ku/\x1a-#Zl\\!LEo\'16EVB3pJ3& g+\\S/H\x1aIFs\x1dNmv\\ZW(i`Xg9P)R](a#GXfUpj-s0!\x1f`)\x1elt)_?4\x1dZDl%Gs.TvQ1(x\\[SOznY5Cl[BLx8@\\&M\x1c\'Nv.B;\x1bgk:L;$v%tpQ\x1fT]\\?-MUcQG=Irnx$fC+!,Kz\x1bzP*G\x1ak"X[\x1e \x1dBF\x1fR_b\x1b=YCO5dlaY@qWi>i@\x1eS5IL4fjc\x1aKDDFT\'HRT@.1A\'

In [12]:
import os
import tempfile

basename = "input.txt"
tempdir = tempfile.mkdtemp()
FILE = os.path.join(tempdir, basename)
print(FILE)

/tmp/tmpk01s5uxn/input.txt


In [13]:
data = fuzzer()
with open(FILE, "w") as f:
    f.write(data)

contents = open(FILE).read()
print(contents)
assert contents == data

-"6*#&%:+&!60>905&/6-"9# 872 4*&7.#&#1*(+!>=>8


In [14]:
import subprocess

program = "bc"
with open(FILE, "w") as f:
    f.write("2+2\n")
result = subprocess.run(
    [program, FILE],
    stdin=subprocess.PIPE,
    stdout=subprocess.PIPE,
    universal_newlines=True,
)

In [15]:
result.stdout

'4\n'

In [16]:
result.returncode

0

In [17]:
result.stderr

In [18]:
trials = 100
program = "bc"

runs = []

for i in range(trials):
    data = fuzzer()
    with open(FILE, "w") as f:
        result = subprocess.run(
            [program, FILE],
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            universal_newlines=True,
        )

        runs.append((data, result))

In [19]:
sum(1 for (data, result) in runs if result.stderr == "")

0

In [20]:
errors = [(data, result) for (data, result) in runs if result.stderr != ""]
(first_data, first_result) = errors[0]
print(repr(first_data))
print(first_result.stderr)

"$9-::9+ 2?!>7#()6(1,?2*>*':=&"
None


In [21]:
[
    result.stderr
    for (data, result) in runs
    if result.stderr
    and "illegal character" not in result.stderr
    and "parse error" not in result.stderr
    and "syntax error" not in result.stderr
]

[]

In [22]:
def crash_if_too_long(s):
    buffer = "Thursday"
    if len(s) > len(buffer):
        raise ValueError

In [23]:
from fuzzingbook.Fuzzer import ExpectError

trials = 100
with ExpectError():
    for i in range(trials):
        s = fuzzer()
        crash_if_too_long(s)

Traceback (most recent call last):
  File "/tmp/ipykernel_23818/2508798955.py", line 7, in <module>
    crash_if_too_long(s)
  File "/tmp/ipykernel_23818/2784561514.py", line 4, in crash_if_too_long
    raise ValueError
ValueError (expected)


In [24]:
def hang_if_n_space(s):
    i = 0
    while True:
        if i < len(s):
            if s[i] == " ":
                break
        i += 1

In [25]:
from fuzzingbook.Fuzzer import ExpectTimeout

trials = 100
with ExpectTimeout(2):
    for _ in range(trials):
        s = fuzzer()
        hang_if_n_space(s)

Traceback (most recent call last):
  File "/tmp/ipykernel_23818/1808899220.py", line 7, in <module>
    hang_if_n_space(s)
  File "/tmp/ipykernel_23818/2837961466.py", line 3, in hang_if_n_space
    while True:
  File "/home/harshil/.pyenvs/fuzzing/lib/python3.11/site-packages/fuzzingbook/Timeout.py", line 147, in timeout_handler
    raise TimeoutError()
TimeoutError (expected)


In [26]:
def collapse_if_too_large(s):
    if int(s) > 1000:
        raise ValueError

In [ ]:
long_number = fuzzer(100, ord("0"), 10)
print(long_number)

0814143445386117485790103234934


In [ ]:
with open("program.c", "w") as f:
    f.write(
        """
#include <stdlib.h>
#include <string.h>

int main(int argc, char** argv) {
    /* Create an array with 100 bytes, initialized with 42 */
    char *buf = malloc(100);
    memset(buf, 42, 100);

    /* Read the N-th element, with N being the first command-line argument */
    int index = atoi(argv[1]);
    char val = buf[index];

    /* Clean up memory so we don't leak */
    free(buf);
    return val;
}
    """
    )

In [ ]:
from fuzzingbook.bookutils import print_file

print_file("program.c")

#include <stdlib.h>
#include <string.h>

int main(int argc, char** argv) {
    /* Create an array with 100 bytes, initialized with 42 */
    char *buf = malloc(100);
    memset(buf, 42, 100);

    /* Read the N-th element, with N being the first command-line argument */
    int index = atoi(argv[1]);
    char val = buf[index];

    /* Clean up memory so we don't leak */
    free(buf);
    return val;
}
    

In [43]:
!clang -fsanitize=address -g -o program program.c

In [44]:
!./program 99; echo $?

42


In [45]:
!./program 110

==25013==ERROR: AddressSanitizer: heap-buffer-overflow on address 0x60b00000015e at pc 0x561c8cfcdf4c bp 0x7ffc7b120260 sp 0x7ffc7b120258
READ of size 1 at 0x60b00000015e thread T0
    #0 0x561c8cfcdf4b in main /home/harshil/concurrency-testing-fuzzing/program.c:12:16
    #1 0x7fa54276b249 in __libc_start_call_main csu/../sysdeps/nptl/libc_start_call_main.h:58:16
    #2 0x7fa54276b304 in __libc_start_main csu/../csu/libc-start.c:360:3
    #3 0x561c8cf10300 in _start (/home/harshil/concurrency-testing-fuzzing/program+0x20300) (BuildId: a7defe265a60f982a1ddea1c01cc23792b1596ff)

0x60b00000015e is located 10 bytes to the right of 100-byte region [0x60b0000000f0,0x60b000000154)
allocated by thread T0 here:
    #0 0x561c8cf9314e in __interceptor_malloc (/home/harshil/concurrency-testing-fuzzing/program+0xa314e) (BuildId: a7defe265a60f982a1ddea1c01cc23792b1596ff)
    #1 0x561c8cfcdebf in main /home/harshil/concurrency-testing-fuzzing/program.c:7:17
    #2 0x7fa54276b249 in __libc_start_call_

In [46]:
!rm -fr program program.*

In [ ]:
secrets = (
    "<space for reply>"
    + fuzzer(100)
    + "<secret-certificate>"
    + fuzzer(100)
    + "<secret-key>"
    + fuzzer(100)
    + "<other-secrets>"
)

In [48]:
uninitialized_memory_marker = "deadbeef"
while len(secrets) < 2048:
    secrets += uninitialized_memory_marker

In [ ]:
def heartbeat(reply: str, length: int, memory: str) -> str:
    # Store reply in memory
    memory = reply + memory[len(reply) :]

    # Send back heartbeat
    s = ""
    for i in range(length):
        s += memory[i]
    return s

In [50]:
heartbeat("potato", 6, memory=secrets)

'potato'

In [52]:
heartbeat("bird", 4, memory=secrets)

'bird'

In [53]:
heartbeat("hat", 500, memory=secrets)

'hatace for reply>>."(2+49"?63+?*2:70-285,\'-*5*7"&><secret-certificate>=8!5 *\'+9:)>5<: >07$"&".00&5)=\' & 89"::!-67\'262\'<\'% ! "*.-?>)1-#6,,/&-#<secret-key>40=0%4&. (>%#:78-79!>81&+2:+*9):6.,# /7120)4*0!=?9"043$))/.,:*,)&9&>,>7\' <other-secrets>deadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdea'

In [ ]:
with ExpectError():
    for _ in range(10):
        s = heartbeat(fuzzer(), random.randint(1, 500), memory=secrets)
        assert not s.find(uninitialized_memory_marker)
        assert not s.find("secret")

Traceback (most recent call last):
  File "/tmp/ipykernel_23818/1920461974.py", line 4, in <module>
    assert not s.find(uninitialized_memory_marker)
AssertionError (expected)


In [ ]:
airport_codes: Dict[str, str] = {
    "YVR": "Vancouver",
    "JFK": "New York-JFK",
    "CDG": "Paris-Charles de Gaulle",
    "CAI": "Cairo",
    "LED": "St. Petersburg",
    "PEK": "Beijing",
    "HND": "Tokyo-Haneda",
    "AKL": "Auckland",
}

In [57]:
airport_codes["YVR"]

'Vancouver'

In [58]:
"AKL" in airport_codes

True

In [61]:
def code_rep_chcker(code: str) -> bool:
    assert len(code) == 3, "Airport code must have three characters" + repr(code)
    for c in code:
        assert c.isalpha(), "Non-Letter in airport code: " + repr(code)
        assert c.isupper(), "Lowercase letter in airport code: " + repr(code)
    return True